In [ ]:
%load_ext autoreload

In [ ]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
#from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from meshcat import Visualizer

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph, CoulombFriction
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics, SpatialInertia, UnitInertia
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform
from pydrake.geometry import Box, Role, GeometrySet,CollisionFilterDeclaration
import meshcat
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer

In [ ]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sos_iris_certifier/assets/bimanual_iiwa.yaml")
table_dim = np.array([4, 4, 0.2])
table = plant.AddRigidBody("table", SpatialInertia(
        mass=1.0, p_PScm_E=np.array([0., 0., 0.]), G_SP_E=UnitInertia(1.0, 1.0, 1.0)))
plant.WeldFrames(plant.world_frame(), table.body_frame(), RigidTransform(p=np.array([0, 0, -table_dim[2]])))
# plant.RegisterVisualGeometry(table, RigidTransform(), Box(*table_dim), "table_vis",
#                              np.array([0.5, 0.5, 0.5, 1.]))
plant.RegisterCollisionGeometry(table, RigidTransform(), Box(*table_dim), "table_collision",
                                CoulombFriction(0.9, 0.8))

directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa_1, wsg_1, iiwa_2, wsg_2, shelf] = models

plant.Finalize()

#viz_role = Role.kIllustration
viz_role = Role.kProximity
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)
diagram = builder.Build()
q0 = [0, -0.2, 0, -1.2, 0, 1.6, 0]
# [-1.5       -1.5       -2.0943951 -2.0943951 -1.5       -1.5
#  -2.0943951 -2.0943951 -1.5       -1.5       -2.0943951 -2.0943951
#  -0.1       -0.1      ]
index = 0
for joint_index in plant.GetJointIndices(iiwa_1.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
index = 0
for joint_index in plant.GetJointIndices(iiwa_2.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
sg_context = scene_graph.GetMyContextFromRoot(context)
diagram.Publish(context)

In [ ]:
filter_manager = scene_graph.collision_filter_manager(sg_context)
inspector = scene_graph.model_inspector()

iiwa1 = [[], [], [], [], [], [], [], []]
iiwa2 = [[], [], [], [], [], [], [], []]
wsg1 = []
wsg2 = []
shelf = []
for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity):
    gid_name = inspector.GetName(gid)
    if "iiwa_1::iiwa_link_" in gid_name:
        link_num = gid_name[18]
        iiwa1[int(link_num)].append(gid)
    if "iiwa_2::iiwa_link_" in gid_name:
        link_num = gid_name[18]
        iiwa2[int(link_num)].append(gid)
    if "wsg_1" in gid_name:
        wsg1.append(gid)
    if "wsg_2" in gid_name:
        wsg2.append(gid)
    if ("shelves::top" in gid_name or "shelves::shelf_lower" in gid_name or
        "shelves::shelf_upper" in gid_name):
        shelf.append(gid)


filter_manager.Apply(CollisionFilterDeclaration().ExcludeWithin(GeometrySet(
        iiwa1[0] + iiwa1[1] + iiwa1[2] + iiwa1[3] + shelf)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[1] + iiwa1[2]+ iiwa1[3]), GeometrySet(iiwa1[4] + iiwa1[5])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[3] + iiwa1[4]), GeometrySet(iiwa1[6])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[2] + iiwa1[3] + iiwa1[4] + iiwa1[5] + iiwa1[6]), GeometrySet(iiwa1[7] + wsg1)))

filter_manager.Apply(CollisionFilterDeclaration().ExcludeWithin(GeometrySet(
        iiwa2[0] + iiwa2[1] + iiwa2[2] + iiwa2[3] + shelf)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[1] + iiwa2[2]+ iiwa2[3]), GeometrySet(iiwa2[4] + iiwa2[5])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[3] + iiwa2[4]), GeometrySet(iiwa2[6])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[2] + iiwa2[3] + iiwa2[4] + iiwa2[5] + iiwa2[6]), GeometrySet(iiwa2[7] + wsg2)))

filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[1]), GeometrySet(iiwa2[0] + iiwa2[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[2]), GeometrySet(iiwa2[0] + iiwa2[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[1]), GeometrySet(iiwa2[2])))

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

# seed  with rrtiris

In [ ]:
import ipywidgets as widgets
from IPython.display import display
q = np.zeros((14,))
sliders = []
for low, up, joint_idx in zip(plant.GetPositionLowerLimits()[:7], plant.GetPositionUpperLimits()[:7], range(7)):
    sliders.append(
        widgets.FloatSlider(
        min=low, 
        max=up, 
        value=0, 
        description=f'iiwa_joint_{joint_idx}'
        )
    )

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)

def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    plant.SetPositions(plant_context, q)
    diagram.Publish(context)
    print(col_func_handle(q), end = '\r')

for idx, slider in enumerate(sliders):
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
for slider in sliders:
    display(slider)
display(vis.jupyter_cell())

In [ ]:
q_start = np.array([-0.06706,  1.4056 ,  1.53294, -1.3944 ,  0.23294, -1.4944 ,
       -1.45433, -0.36706,  0.2056 ,  0.     , -1.3944 ,  0.     ,
        0.     ,  0.     ])
q_end = np.array([ 0.63294,  0.1056 ,  0.     , -1.5944 , -1.66706,  0.6056 ,
       -1.45433,  0.13294,  1.8056 ,  1.83294,  1.6056 ,  0.33294,
        1.5056 ,  1.24567])

In [ ]:
plant.SetPositions(plant.GetMyMutableContextFromRoot(context),q_start)
diagram.Publish(context)

In [ ]:
col_func_handle(q)

In [ ]:
q = q_start

In [ ]:
q

In [ ]:
q

In [ ]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False


In [ ]:
plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q_start)
diagram.Publish(context)
#r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
 #                                             iris_options)

In [ ]:
q_end = np.array([ 0.63294,  0.1056 ,  0.     , -1.5944 , -1.66706,  0.6056 ,
       -1.45433,  0.13294,  1.8056 ,  1.83294,  1.6056 ,  0.33294,
        1.5056 ,  1.24567])

In [ ]:
q_end

In [ ]:
start = man_seed_points_mark[15, :]
goal = man_seed_points_mark[15, :]  

In [ ]:
angs1 = [ 0.6, 1.7, 0, 0, 0, 0, 0]
angs2 = [ 0, 0, 0, 0, 0, 0, 0]
start = np.array(angs1 + angs2)
plant.SetPositions(plant_context, q_end)
diagram.Publish(context)

In [ ]:
man_seed_points_mark = np.array([[0, 0, -0.2, -0.2, 0, 0, -1.2, -1.2, 0, 0, 1.6, 1.6, 0, 0], # both in free
                        [8.28832821e-01, 0, 3.89848505e-01, -0.2, -7.90395431e-01, 0, -1.37365375,
                         -1.2, 1.74120864e-01, 0, 1.38859053e-03, 1.6, 0.1, 0], # 2 in free, 1 in top
                        [8.28832821e-01, -7.95441561e-01, 3.89848505e-01, 4.75848165e-01, -7.90395431e-01,
                         1.00448034, -1.37365375, -8.86131742e-01, 1.74120864e-01, -5.11686194e-01,
                         1.38859053e-03, 2.89610812e-01, 0.1, 0.1], # 2 at [0.7, 0.35, 0.9]
                        [8.28832821e-01, -6.10262076e-01, 3.89848505e-01, 6.66352604e-01, -7.90395431e-01,
                         4.22655432e-01, -1.37365375, -1.72823980, 1.74120864e-01, -4.44800260e-01,
                         1.38859053e-03, -6.56452038e-01, 0.1, 0.1], # 2 at [0.7, 0.35, 0.4]
                        [8.28832821e-01, -6.82228174e-01, 3.89848505e-01, 5.51697498e-01, -7.90395431e-01,
                         4.47205935e-01, -1.37365375, -2.04405442, 1.74120864e-01, -2.42663861e-01,
                         1.38859053e-03, -9.77773546e-01, 0.1, -0.1], # 2 at [0.6, 0.35, 0.4]
                        [8.28832821e-01, 2.26229695e-01, 3.89848505e-01, -2.96365086e-02, -7.90395431e-01,
                         -6.21371127e-01, -1.37365375, -2.09439510, 1.74120864e-01, -1.07835208,
                         1.38859053e-03, -2.98578277e-01, 0.1, 0.9], # 2 at [0.5, 0.35, 0.4]
                        [8.28832821e-01, -2.09842199, 3.89848505e-01, -1.30002031e-01, -7.90395431e-01,
                         1.39943233, -1.37365375, -1.04642463, 1.74120864e-01, -1.93733232, 1.38859053e-03,
                         -7.50304996e-01, 0.1, -8.89281286e-01], # 2 at [0.5, 0.35, 1]
                        [8.28832821e-01, -1.91937039e-01, 3.89848505e-01, 4.75052180e-01, -7.90395431e-01,
                         -1.89946110e-01, -1.37365375, -2.02253834, 1.74120864e-01, -4.56413951e-01,
                         1.38859053e-03, -9.13513888e-01, 0.1, 0.4], # 2 at [0.6, 0.35, 0.4]
                        [1.02177876, 2.28521279e-01, 5.05308783e-01, 1.49635475, -1.09566951, -1.60472828,
                         -1.37368406, -1.81080834, 3.66017349e-01, 1.50138450, 1.36560389e-03, 1.38491810e-03,
                         0.1, 0.1],
                        [8.28832821e-01, -9.54010087e-02, 3.89848505e-01, 7.30225069e-01, -7.90395431e-01,
                         -7.50405052e-01, -1.37365375, -1.79905247, 1.74120864e-01, 1.33241194,
                         1.38859053e-03, -7.40624016e-01, 0.1, -0.57],
                        [1.22118727, 2.27421758e-01, 8.94352120e-01, 1.33098255, -1.43293457, -1.52371213,
                         -1.14371009, -1.81090526, 9.79242371e-01, 1.42840943, 9.93723820e-02,
                         -1.40727990e-03, -0.1, -0.1],
                        [1.19160032, 0.22785254, 0.82375722, 1.3453217, -1.3082158, -1.53542116,
                         -1.08385523, -1.82146975, 0.175, -0.364, 0.21723784, -0.03523606,
                         0.69034143, 1.74316121],
                        [1.19160031, -0.0310574, 0.82375722, 2.03218165, -1.3082158, -1.48258324,
                         -1.08385523, -1.40024396, 0.175, -0.175, 0.21723784, 0.15989148,
                         0.69034143, 2.13763237],
                        [0.43694414, -0.61026208, 1.39932452, 0.6663526, 0.65263265, 0.42265543,
                         -1.64261262, -1.7282398, -0.92382883, -0.44480026, -0.89065069, -0.65645204,
                         -0.21, 0.1],
                        [0.34738906, -0.61026208, 1.33779899, 0.6663526, 0.40099608, 0.42265543,
                         -1.63794945, -1.7282398, -1.08798551, -0.44480026, -1.17689931, -0.65645204,
                         -0.1, 0.1],
                        [0.30936788, -0.61026208, 1.26775351, 0.6663526, 0.10431461, 0.42265543,
                         -1.50188975, -1.7282398, -1.34442298, -0.44480026, -1.30030757, -0.65645204,
                         0.29, 0.1],
                        [-0.65931491, -0.61026208, 1.77100716, 0.6663526, 1.00938531, 0.42265543,
                         -1.92035289, -1.7282398, -2.16413347, -0.44480026, -0.40218662, -0.65645204,
                         0.15, 0.1],
                        [-1.27038293, -0.61026208, 0.8106163, 0.6663526, 1.04863723, 0.42265543,
                         -2.0271749, -1.7282398, -2.25291433, -0.44480026, -1.27975529, -0.65645204,
                         0.78, 0.1],
                        [-1.76741257, -0.61026208, 0.90778111, 0.6663526, 1.02982481, 0.42265543,
                         -1.83973807, -1.7282398, -2.17572169, -0.44480026, -1.13224798, -0.65645204,
                         0.84, 0.1],
                        [0, -0.61026208, -0.2, 0.6663526, 0, 0.42265543, -1.2, -1.7282398, 0,
                         -0.44480026, 1.6, -0.65645204, 0, 0.1],
                        [0, -0.61026208, -0.2, 0.5, 0, 0.3, -1.2, -1.95, 0,
                         -0.44480026, 1.6, -0.65645204, 0, 0.1],
                        [0, -0.61026208, -0.2, 0.3, 0, 0.23, -1.2, -2.05, 0,
                         -0.44480026, 1.6, -0.8, 0, 0.1],
                        [1.1, 0, 0., -0.2, -7.90395431e-01, 0, -1.8,
                         -1.2, -0.15, 0, -0.2, 1.6, 0.1, 0] # [0.7, 0.15, 0.7]

                        ])

# Build Plant


In [ ]:
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
locs = [[0.,0.,0.],[0.,0.,0.]]
for idx, model in enumerate(models):
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    
plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# Setup Meshcat


In [ ]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)
visualizer.jupyter_cell()

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

# Setup IRIS Options and Generate Regions

In [ ]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 0.1e-5
iris_options.max_faces_per_collision_pair = 30
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

seed_points_q = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])    # passing
seed_points_q = seed_points_q[:, [0,2,1]]
seed_points = []
seed_points_q2 = []
for idx in range(seed_points_q.shape[0]):
    seed_points.append(Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)) ))
    seed_points_q2.append(Ratfk.ComputeQValue(seed_points[-1], np.zeros(3,)))
seed_points = np.array(seed_points)
seed_points_q2 = np.array(seed_points_q2)

In [ ]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if False:
        #starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    print(f"{r.A()}t <= {r.b()}")
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

In [ ]:
visualizer.plot_regions(regions, ellipses=None, region_suffix='', opacity = 0.8)
visualizer.plot_seedpoints(seed_points)

In [ ]:
visualizer.plot_regions([regions[3]], ellipses=None, region_suffix='', opacity = 1.0)

In [ ]:
seed_points

In [ ]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
# from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, prm, utils, rrtiris
from pydrake.all import (MathematicalProgram, Variable, HPolyhedron, le, SnoptSolver, Solve) 


In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

In [ ]:
def meshcat_line(x_start, x_end, width):
    x_end_shift = x_end.copy()
    x_end_shift[0:2] += width
    x_start_shift = x_start.copy()
    x_start_shift[1:2] += width
    
    points = np.array([[x_start, x_end, x_end_shift, x_start_shift]]).reshape(-1,3)
    triangles = np.array([[0,1,2],[0,2,3]]).reshape(-1,3)
    mc_geom = meshcat.geometry.TriangularMeshGeometry(points, triangles)
    return mc_geom

In [ ]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 50

In [ ]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

# def eval_cons(q0, q1, q2, c, tol):
#     return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
def eval_cons(q0, q1, q2, c, tol):
    in_collision = 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    q = np.array([q0, q1, q2])
    lo = 0.9*np.array(q_low)
    hi = 0.9*np.array(q_high)
    lowercol = np.sum(1-1*(q<= hi))
    uppercol = np.sum(1-1*(q>= lo))
    return in_collision or lowercol or uppercol
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))from t_space_utils import convert_q_to_t
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


In [ ]:
#marching cubes
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

# V-HACD Setup

In [ ]:
import trimesh
mesh = trimesh.Trimesh(vertices, triangles)
decomp = trimesh.interfaces.vhacd.convex_decomposition(mesh)
for it, region in enumerate(decomp):
    rand = np.random.rand(3)
    rand = (255*rand).astype(int)
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex(tuple(rand.tolist())))
    mat.opacity = 0.5
    verts = np.array(region.vertices)
    faces = np.array(region.faces)
    vis2["v-hacd"][str(it)].set_object(
            meshcat.geometry.TriangularMeshGeometry(verts, faces),
            mat)

In [ ]:
# HRep of V-HACD regions
from pypoman import compute_polytope_halfspaces
vhacd_regions = []
for region in decomp:
    A, b = compute_polytope_halfspaces(region.vertices)
    print('A shape:', A.shape, ' trimesh faces', region.faces.shape)
    vhacd_regions.append(HPolyhedron(A=A, b=b))

In [ ]:
# VRep of V-HACD regions
from pydrake.all import VPolytope
vhacd_regions_vrep = [VPolytope(region.vertices) for region in decomp]

In [ ]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

In [ ]:
#SPP + IRIS CONFIG
start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])
#
#plot start and target
mat = meshcat.geometry.MeshLambertMaterial(color=0xFFDD36)
mat.reflectivity = 1.0
vis2['start'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['start'].set_transform(
                meshcat.transformations.translation_matrix(start.reshape(-1,)))

mat = meshcat.geometry.MeshLambertMaterial(color=0x06D300)
mat.reflectivity = 1.0
vis2['target'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['target'].set_transform(
                meshcat.transformations.translation_matrix(target.reshape(-1,)))

In [ ]:
def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = False)

def plot_callback(region, seed_point, pos_samp, region_id, vis):
    scale = np.clip(150 + region_id *105/5.0, a_min = 0, a_max = 255)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((int(0.2*scale), 155, int(scale))) , wireframe=True)
    mat_seed = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((0, 0, 0)), wireframe=True)
    mat_samp = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((155, 0, 155)), wireframe=True)
    mat.opacity = 0.2
    
    #print('seed_point', seed_point)
    #print('pos_samp', pos_samp)
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis['regions'],
                       name = str(region_id),
                       mat = mat)
    utils.plot_point(seed_point,
                     radius = 0.05, 
                     mat = mat_seed, 
                     vis = vis['seed'], 
                     marker_id = str(region_id))
    
    utils.plot_point(pos_samp,
                     radius = 0.05, 
                     mat = mat_samp, 
                     vis = vis['samples'], 
                     marker_id = str(region_id))
    
plot_callback_handle = partial(plot_callback, vis = vis2['rrtiris']) 

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrti_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

def get_closest_point_in_set(polys, sample, dim = 3):
    num_regions = len(polys) 
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(dim*num_regions, 'x')  
    cost = x - np.tile(sample, (1, num_regions)).squeeze()
    cost = cost@cost.T
    prog.AddCost(cost)
    for idx in range(num_regions):
        A = polys[idx].A()
        b = polys[idx].b()
        prog.AddConstraint(le(A@x[idx*dim:(idx + 1)*dim], b))
    solver = SnoptSolver()
    result = solver.Solve(prog)
    x_sol = result.GetSolution(x).reshape(num_regions,3)
    dists = np.linalg.norm(x_sol-sample, axis = 1)
    closest = np.argmin(dists)
    min_dist = dists[closest]
    min_point = x_sol[closest, :]
    
    return min_point, min_dist, x_sol, result

In [ ]:
RRTI = rrtiris.RRTIRIS(start = start, 
                       goal = target , 
                       limits = [np.array(q_low), np.array(q_high)], 
                       iris_handle = iris_hand,
                       col_func_handle = rrti_col_fn_handle, 
                       init_goal_sample_rate = 0.05,
                       goal_sample_rate_scaler = 0.6,
                       verbose = True,
                       plotcallback = plot_callback_handle,
                       sample_collision_free = True
                       )

In [ ]:
RRTI.run(40)

In [ ]:
regions = RRTI.node_regions

In [ ]:
#using hand-crafted seed points

seed_points = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])    # passing


iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    ellipses.append(ellipse)
    regions.append(hpoly)

In [ ]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

q_low_np = np.array(q_low)
q_high_np = np.array(q_high)

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 6
seed_points = [start, target]

for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    regions.append(reg)
    ellipses.append(ell)
    
for _ in range(its):
    #rejection sampling to get initial feasible point 
    found = False
    while not found:
        t = np.random.rand(3)
        q_samp = (1-t)*q_low_np + t*q_high_np
        found = (col_func_handle(*q_samp)==0.0)
    print("point found: ", q_samp)
    ell, reg = do_iris(q_samp, iris_options)
    if ell.Volume() < 10.0:
        regions.append(reg)
        ellipses.append(ell)
    

In [ ]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))

for region in regions:
    A = region.A()
    b = region.b()
    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 50, 50, 50, col_hand, 0.5)
    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
    C = ellipses[idx].A()
    d = ellipses[idx].center()
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.15
    vis2['ellipses'][ellstr].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1

In [ ]:
# Solve path planning

start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity([.8, .8, .8])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj = spp.SolveVerbose()
print(time.time() - start_time)
print(traj.start_time())
print(traj.end_time())
#visualize
maxit = 60
pts = []
for it in range(maxit):
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    vis2['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.06), mat)
    vis2['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
    
    set_joint_angles(pts[-1].reshape(-1,))
    tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
    R_l2 = tf_l2.rotation()
    tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
    
    tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
    R_la = tf_la.rotation()
    tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
     
    
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['link2']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['link2']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_l2))
    mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
    mat.reflectivity = 1.0
    vis['traj']['linka']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['linka']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_la))
    

In [ ]:
spp.Solve

In [ ]:
#rrt cfg
limits = [np.array(q_low), np.array(q_high)]
mat = meshcat.geometry.MeshLambertMaterial(color= 0x000000 , wireframe=True)

def plotting_fn(parent, child, pos_samp, idx, mat, width):
    vis2['rrt']['tree']['line' + str(idx)].set_object( meshcat_line(parent.pos, child.pos, width), mat)
    
plotting_fn_handle = partial(plotting_fn, mat = mat, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrt_col_fn_handle = partial(collision, col_func_handle = col_func_handle)
    
RRT = rrt.RRT(start = start,
              goal = target,
              limits = limits,
              col_func_handle=rrt_col_fn_handle,
              max_extend_length=0.1,
              extend_steps=0.002,
              init_goal_sample_rate=0.1,
              goal_sample_rate_scaler=0.9,
              verbose = True, 
              plotcallback = plotting_fn_handle,
              sample_collision_free= False)

success, path = RRT.run(8000)
if success:
    mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
    mat.wireframeLinewidth = 2.0
    num_waypoints = len(path)
    for idx in range(num_waypoints-1):
        vis2['rrt']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
    traj= utils.PWLinTraj(path, 5.0)

In [ ]:
#prm cfg
limits = [np.array(q_low), np.array(q_high)]
mat = meshcat.geometry.MeshLambertMaterial(color= 0x000000 , wireframe=True)

def plotting_fn(nodes, adjacency_list, mat, width):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = nodes[node_idx, :]
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = nodes[adjacency_list[node_idx][edge_idx], :]
            vis2['prm']['rm']['line' + str(plt_idx)].set_object( meshcat_line(pos1, pos2, width), mat)
            plt_idx +=1
            
plotting_fn_handle = partial(plotting_fn, mat = mat, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])

prm_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

PRM = prm.PRM( 
            limits,
            num_points = 400,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

PRM.add_start_end(start, target)
PRM.plot()
path, sp_length = PRM.find_shortest_path()

mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
mat.wireframeLinewidth = 2.0
num_waypoints = len(path)
for idx in range(num_waypoints-1):
    vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
traj= utils.PWLinTraj(path, 5.0)

In [ ]:
#loop
substeps = 1000
utils.animate(traj, showres, substeps, 10*substeps)
            

In [ ]:
def get_AABB_limits(hpoly):
    #im using snopt, sue me
    max_limits = []
    min_limits = []
    A = hpoly.A()
    b = hpoly.b()

    for idx in range(3):
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(3, 'x')
        cost = x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        min_limits.append(result.get_optimal_cost()-0.01)
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(3, 'x')
        cost = -x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        max_limits.append(-result.get_optimal_cost() + 0.01)
    return max_limits, min_limits

def plot_3d_poly(region, resolution, vis, name, mat = None):
    
    def inpolycheck(q0,q1,q2, A, b):
        q = np.array([q0, q1, q2])
        res = np.min(1.0*(A@q-b<=0))
        #print(res)
        return res
    
    aabb_max, aabb_min = get_AABB_limits(region)
    print('AABB:', aabb_min, aabb_max)
    col_hand = partial(inpolycheck, A=region.A(), b=region.b())
    vertices, triangles = mcubes.marching_cubes_func(tuple(aabb_min), 
                                                     tuple(aabb_max),
                                                     resolution, 
                                                     resolution, 
                                                     resolution, 
                                                     col_hand, 
                                                     0.5)
    if mat is None:
        mat = meshcat.geometry.MeshLambertMaterial(color=0x000000 , wireframe=True)
        mat.opacity = 0.3
    vis[name].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)

def plot_point(loc, radius, mat, vis, marker_id):
    vis['markers'][marker_id].set_object(
                meshcat.geometry.Sphere(radius), mat)
    vis['markers'][marker_id].set_transform(
                meshcat.transformations.translation_matrix(loc))

def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = True)

def get_closest_point_in_set(polys, sample, dim = 3):
    num_regions = len(polys) 
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(dim*num_regions, 'x')  
    cost = x - np.tile(sample, (1, num_regions)).squeeze()
    cost = cost@cost.T
    prog.AddCost(cost)
    for idx in range(num_regions):
        A = polys[idx].A()
        b = polys[idx].b()
        prog.AddConstraint(le(A@x[idx*dim:(idx + 1)*dim], b))
    solver = SnoptSolver()
    result = solver.Solve(prog)
    x_sol = result.GetSolution(x).reshape(num_regions,3)
    dists = np.linalg.norm(x_sol-sample, axis = 1)
    closest = np.argmin(dists)
    min_dist = dists[closest]
    min_point = x_sol[closest, :]
    
    return min_point, min_dist, x_sol, result

In [ ]:
seed_points = []
polys = []
ells = []
region, ell = iris_hand(start)
polys.append(region)
ells.append(ell)
region, ell = iris_hand(target)
polys.append(region)
ells.append(ell)
idx = 0
marker_mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((255, 125, 125)) , wireframe=True)
for poly in polys:
    plot_3d_poly(poly, resolution= 20, vis = vis2['irisrrt'], name = 'poly_' + str(idx), mat = None )
    #plot_point(, radius = 0.05, mat = marker_mat , vis =vis2['irisrrt'], marker_id = str(idx))
    idx +=1

In [ ]:
sample = np.array([0.2, -0.2, 0.3])
seed_points.append(sample)
marker_mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((255, 10, 50)) , wireframe=True)

plot_point(sample, radius = 0.05, mat = marker_mat , vis = vis2['irisrrt'], marker_id = 'sample')
new_seed_point, dists, x_sol, result =  get_closest_point_in_set(polys, sample)


marker_mat = meshcat.geometry.MeshLambertMaterial(color= rgb_to_hex((255, 0, 255)) , wireframe=True)
for idx in range(x_sol.shape[0]): 
    plot_point(x_sol[idx,:], radius = 0.05, mat = marker_mat , vis = vis2['irisrrt'], marker_id = 'sol' + str(idx))

region, ell = iris_hand(new_seed_point)
polys.append(region)
ells.append(ell)
plot_3d_poly(polys[-1], resolution= 20, vis = vis2['irisrrt'], name = 'poly_' + str(len(polys)), mat = None )
    

In [ ]:
result.is_success()

In [ ]:
def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = True)

def plot_callback(region, seed_point, pos_samp, region_id, vis):
    scale = np.clip(150 + region_id *105/5.0, a_min = 0, a_max = 255)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((int(0.2*scale), 0, int(scale))) , wireframe=True)
    mat_seed = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((0, 0, 0)), wireframe=True)
    mat_samp = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((155, 0, 155)), wireframe=True)
    
    print('seed_point', seed_point)
    print('pos_samp', pos_samp)
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis['regions'],
                       name = str(region_id),
                       mat = mat)
    utils.plot_point(seed_point,
                     radius = 0.05, 
                     mat = mat_seed, 
                     vis = vis['seed'], 
                     marker_id = str(region_id))
    
    utils.plot_point(pos_samp,
                     radius = 0.05, 
                     mat = mat_samp, 
                     vis = vis['samples'], 
                     marker_id = str(region_id))
    
plot_callback_handle = partial(plot_callback, vis = vis2['rrtiris']) 

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrti_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

In [ ]:
RRTI = rrtiris.RRTIRIS(start = start, 
                       goal = target , 
                       limits = [np.array(q_low), np.array(q_high)], 
                       iris_handle = iris_hand,
                       col_func_handle = rrti_col_fn_handle, 
                       init_goal_sample_rate = 0.05,
                       goal_sample_rate_scaler = 0.1,
                       verbose = False,
                       plotcallback = plot_callback_handle,
                       sample_collision_free = True
                       )

In [ ]:
RRTI.run(12)

In [ ]:
cost.